<a href="https://colab.research.google.com/github/SaiNat123/projet_9/blob/main/notebook_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,149 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,224 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,438 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,499 kB]


## Affichage des informations sur la session en cours et liens vers Spark UI

In [2]:
%%info

UsageError: Cell magic `%%info` not found.


## Import des librairies

In [22]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA, StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT

## Définition des PATH pour charger les images et enregistrer les résultats

In [10]:
PATH = 'sample_data'
PATH_Data = PATH+'/Images'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

PATH:        sample_data
PATH_Data:   sample_data/Images
PATH_Result: sample_data/Results


## Traitement des données
### Chargement des données

In [12]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [17]:
images.show(20)

+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/content/sam...|2024-09-09 17:52:...|  4893|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4875|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4841|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4830|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4371|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4346|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4340|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4339|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4281|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4235|[FF D8 FF E0 00 1...|
|file:/content/sam...|2024-09-09 17:52:...|  4201|[FF D8 FF E0 0

In [18]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-----------------------------------------------------+-------+
|path                                                 |label  |
+-----------------------------------------------------+-------+
|file:/content/sample_data/Images/Avocado/r_4_100.jpg |Avocado|
|file:/content/sample_data/Images/Avocado/r_5_100.jpg |Avocado|
|file:/content/sample_data/Images/Avocado/r_41_100.jpg|Avocado|
|file:/content/sample_data/Images/Avocado/r_40_100.jpg|Avocado|
|file:/content/sample_data/Images/Avocado/87_100.jpg  |Avocado|
+-----------------------------------------------------+-------+
only showing top 5 rows

None


### Préparation du modèle

In [19]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [20]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

In [27]:
from pyspark import SparkContext, SparkConf

# Create a Spark configuration
conf = SparkConf().setAppName("BroadcastModelWeights").setMaster("local[*]")

# Initialize SparkContext
sc = SparkContext.getOrCreate(conf=conf)

In [28]:
brodcast_weights = sc.broadcast(new_model.get_weights())

In [29]:
new_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [30]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

### Featurisation

In [31]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/functions.py:389: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [32]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

In [33]:
features_df.dtypes

[('path', 'string'), ('label', 'string'), ('features', 'array<float>')]

In [34]:
features_df.show()

+--------------------+-------+--------------------+
|                path|  label|            features|
+--------------------+-------+--------------------+
|file:/content/sam...|Avocado|[1.3703401, 0.0, ...|
|file:/content/sam...|Avocado|[0.6303392, 0.0, ...|
|file:/content/sam...|Avocado|[0.36889577, 0.0,...|
|file:/content/sam...|Avocado|[0.83358765, 0.0,...|
|file:/content/sam...|Avocado|[0.43423674, 0.0,...|
|file:/content/sam...|Avocado|[0.27026623, 0.0,...|
|file:/content/sam...|Apricot|[0.22484958, 0.38...|
|file:/content/sam...|Avocado|[0.8746615, 0.0, ...|
|file:/content/sam...|Apricot|[0.12042565, 0.24...|
|file:/content/sam...|Avocado|[0.355869, 0.0, 0...|
|file:/content/sam...|Apricot|[0.18892503, 0.25...|
|file:/content/sam...|Avocado|[0.08634296, 0.0,...|
|file:/content/sam...|Apricot|[0.22149915, 0.30...|
|file:/content/sam...|Avocado|[0.8457069, 0.0, ...|
|file:/content/sam...|Apricot|[0.30879426, 0.25...|
|file:/content/sam...|Apricot|[0.38278198, 0.37...|
|file:/conte

## Transformation de l'array des features en Vector

In [35]:
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

In [36]:
df_with_vectors = features_df.select(features_df["path"],
                                     features_df["label"],
                                     list_to_vector_udf(features_df["features"]).alias("features"))

In [37]:
df_with_vectors.show()

+--------------------+-------+--------------------+
|                path|  label|            features|
+--------------------+-------+--------------------+
|file:/content/sam...|Avocado|[1.37034010887146...|
|file:/content/sam...|Avocado|[0.63033920526504...|
|file:/content/sam...|Avocado|[0.36889576911926...|
|file:/content/sam...|Avocado|[0.83358764648437...|
|file:/content/sam...|Avocado|[0.43423673510551...|
|file:/content/sam...|Avocado|[0.27026623487472...|
|file:/content/sam...|Apricot|[0.22484958171844...|
|file:/content/sam...|Avocado|[0.87466150522232...|
|file:/content/sam...|Apricot|[0.12042564898729...|
|file:/content/sam...|Avocado|[0.35586899518966...|
|file:/content/sam...|Apricot|[0.18892502784729...|
|file:/content/sam...|Avocado|[0.08634296059608...|
|file:/content/sam...|Apricot|[0.22149914503097...|
|file:/content/sam...|Avocado|[0.84570688009262...|
|file:/content/sam...|Apricot|[0.30879426002502...|
|file:/content/sam...|Apricot|[0.38278198242187...|
|file:/conte

## Standardisation

In [38]:
scaler = StandardScaler(
    inputCol = 'features',
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(df_with_vectors)

df_scaled = scaler.transform(df_with_vectors)
df_scaled.show(6)

+--------------------+-------+--------------------+--------------------+
|                path|  label|            features|      scaledFeatures|
+--------------------+-------+--------------------+--------------------+
|file:/content/sam...|Avocado|[1.37034010887146...|[2.79244743409857...|
|file:/content/sam...|Avocado|[0.63033920526504...|[0.53736600601588...|
|file:/content/sam...|Avocado|[0.36889576911926...|[-0.2593576661758...|
|file:/content/sam...|Avocado|[0.83358764648437...|[1.15674604034655...|
|file:/content/sam...|Avocado|[0.43423673510551...|[-0.0602373702809...|
|file:/content/sam...|Avocado|[0.27026623487472...|[-0.5599216652889...|
+--------------------+-------+--------------------+--------------------+
only showing top 6 rows



## Réduction de dimension

In [39]:
pca = PCA(k=2, inputCol="scaledFeatures", outputCol="pcaFeatures")
model = pca.fit(df_scaled)
result = model.transform(df_scaled)
print('Explained Variance Ratio', model.explainedVariance.toArray())
result.show(10)

Explained Variance Ratio [0.34285176 0.17958196]
+--------------------+-------+--------------------+--------------------+--------------------+
|                path|  label|            features|      scaledFeatures|         pcaFeatures|
+--------------------+-------+--------------------+--------------------+--------------------+
|file:/content/sam...|Avocado|[1.37034010887146...|[2.79244743409857...|[-25.300619635025...|
|file:/content/sam...|Avocado|[0.63033920526504...|[0.53736600601588...|[-12.565358676572...|
|file:/content/sam...|Avocado|[0.36889576911926...|[-0.2593576661758...|[-12.816681254651...|
|file:/content/sam...|Avocado|[0.83358764648437...|[1.15674604034655...|[-11.666593338813...|
|file:/content/sam...|Avocado|[0.43423673510551...|[-0.0602373702809...|[-11.252211196877...|
|file:/content/sam...|Avocado|[0.27026623487472...|[-0.5599216652889...|[-12.200418486244...|
|file:/content/sam...|Apricot|[0.22484958171844...|[-0.6983245382574...|[21.7974888610783...|
|file:/cont

### Rappel du PATH où seront inscrits les fichiers au format "parquet" :

In [40]:
print(PATH_Result)

sample_data/Results


### Enregistrement des données traitées au format "parquet" :

In [41]:
result.write.mode("overwrite").parquet(PATH_Result+'/parquet')

### Enregistrement des données traitées au format "csv" :

In [42]:
pca_df = result.toPandas()

In [43]:
pca_df

,path,label,features,scaledFeatures,pcaFeatures
0,file:/content/sample_data/Images/Avocado/r_40_...,Avocado,"[1.37034010887146, 0.0, 0.30703338980674744, 0...","[2.7924474340985714, -0.8262988907646496, 2.33...","[-25.30061963502559, 13.100642804334447]"
1,file:/content/sample_data/Images/Avocado/88_10...,Avocado,"[0.6303392052650452, 0.0, 0.0, 0.0, 0.17383524...","[0.5373660060158875, -0.8262988907646496, -0.6...","[-12.565358676572927, -20.842967140144836]"
2,file:/content/sample_data/Images/Avocado/60_10...,Avocado,"[0.3688957691192627, 0.0, 0.06557036191225052,...","[-0.25935766617580747, -0.8262988907646496, 0....","[-12.816681254651472, -19.982937552257646]"
3,file:/content/sample_data/Images/Avocado/73_10...,Avocado,"[0.833587646484375, 0.0, 0.014851774089038372,...","[1.1567460403465546, -0.8262988907646496, -0.4...","[-11.666593338813671, -16.106181748030416]"
4,file:/content/sample_data/Images/Avocado/87_10...,Avocado,"[0.4342367351055145, 0.0, 0.000783793802838772...","[-0.06023737028093963, -0.8262988907646496, -0...","[-11.252211196877449, -18.823092749084115]"
5,file:/content/sample_data/Images/Avocado/59_10...,Avocado,"[0.27026623487472534, 0.0, 0.15925666689872742...","[-0.5599216652889746, -0.8262988907646496, 0.9...","[-12.200418486244194, -18.40693427163515]"
6,file:/content/sample_data/Images/Apricot/61_10...,Apricot,"[0.22484958171844482, 0.3805999457836151, 0.0,...","[-0.6983245382574572, 1.1358110689573737, -0.6...","[21.79748886107837, 4.962607492584365]"
7,file:/content/sample_data/Images/Avocado/74_10...,Avocado,"[0.8746615052223206, 0.0, 0.22236374020576477,...","[1.281914665909338, -0.8262988907646496, 1.518...","[-12.893081296184105, -14.765759336113831]"
8,file:/content/sample_data/Images/Apricot/44_10...,Apricot,"[0.12042564898729324, 0.24638187885284424, 0.0...","[-1.0165464082602307, 0.44387558967796503, -0....","[19.192649866385477, 2.9914050208611624]"
9,file:/content/sample_data/Images/Avocado/r_4_1...,Avocado,"[0.35586899518966675, 0.0, 0.21095365285873413...","[-0.29905550412883786, -0.8262988907646496, 1....","[-27.610487652551793, 26.491535070730386]"


In [44]:
pca_df.to_csv(PATH_Result+'/'+'result.csv')

## Chargement des données enregistrées et validation du résultat

In [45]:
dfs = spark.read.parquet(PATH_Result+'/parquet')

In [46]:
df = dfs.toPandas()

## On affiche les 5 premières lignes du DataFrame

In [47]:
df.head(5)

,path,label,features,scaledFeatures,pcaFeatures
0,file:/content/sample_data/Images/Avocado/74_10...,Avocado,"[0.8746615052223206, 0.0, 0.22236374020576477,...","[1.281914665909338, -0.8262988907646496, 1.518...","[-12.893081296184105, -14.765759336113831]"
1,file:/content/sample_data/Images/Apricot/44_10...,Apricot,"[0.12042564898729324, 0.24638187885284424, 0.0...","[-1.0165464082602307, 0.44387558967796503, -0....","[19.192649866385477, 2.9914050208611624]"
2,file:/content/sample_data/Images/Avocado/59_10...,Avocado,"[0.27026623487472534, 0.0, 0.15925666689872742...","[-0.5599216652889746, -0.8262988907646496, 0.9...","[-12.200418486244194, -18.40693427163515]"
3,file:/content/sample_data/Images/Apricot/61_10...,Apricot,"[0.22484958171844482, 0.3805999457836151, 0.0,...","[-0.6983245382574572, 1.1358110689573737, -0.6...","[21.79748886107837, 4.962607492584365]"
4,file:/content/sample_data/Images/Avocado/r_41_...,Avocado,"[0.8457068800926208, 0.0, 0.23516666889190674,...","[1.193678236644367, -0.8262988907646496, 1.641...","[-22.199314882806398, 10.71137948536686]"


### On vérifie la taille du vecteur de caractéristiques des images après réduction à 2 dimensions

In [48]:
df.loc[0,'pcaFeatures'].shape

(2,)